In [ ]:
# !pip install pretrainedmodels==0.7.4
# !pip install efficientnet-pytorch==0.6.3
# !pip install timm==0.3.2

In [1]:
!nvidia-smi

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import SimpleITK as sitk
import cv2
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('/workspace/sunggu'))
sys.path.append(os.path.abspath('/workspace/sunggu/MONAI'))
sys.path.append(os.path.abspath('/workspace/sunggu/4.Dose_img2img'))
sys.path.append(os.path.abspath('/workspace/sunggu/4.Dose_img2img/pix2pixHD-master'))

from sunggu_utils import check_value, take_list, plot_confusion_matrix, list_sort_nicely, find_dir, plot_3D

Tue Mar 30 08:21:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           On   | 00000000:1B:00.0 Off |                  N/A |
| 41%   31C    P8    16W / 280W |      1MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           On   | 00000000:1C:00.0 Off |                  N/A |
| 41%   

# Train

In [ ]:
cd /workspace/sunggu/4.Dose_img2img/pix2pixHD-master

In [ ]:
--continue_train

In [ ]:
!python train.py --name sunggu --gpu_ids 0,1 --label_nc 0 --loadSize 512 --fineSize 512 --input_nc 1 --output_nc 1\
--no_instance --niter_decay 300 --niter 300 --batchSize 16 --phase train --checkpoints_dir /workspace/sunggu/4.Dose_img2img/pix2pixHD-master/runs 

# Test

In [ ]:
cd /workspace/sunggu/4.Dose_img2img/pix2pixHD-master

In [ ]:
!python test.py --name sunggu --gpu_ids 0,1 --label_nc 0 --loadSize 512 --fineSize 512 --input_nc 1 --output_nc 1 \
--no_instance --phase test --checkpoints_dir /workspace/sunggu/4.Dose_img2img/pix2pixHD-master/runs --results_dir /workspace/sunggu/4.Dose_img2img/pix2pixHD-master/runs/result

In [2]:
def diagnose_network(net, name='network'):
    """Calculate and print the mean of average absolute(gradients)
    Parameters:
        net (torch network) -- Torch network
        name (str) -- the name of the network
    """
    mean = 0.0
    count = 0
    for param in net.parameters():
        if param.grad is not None:
            mean += torch.mean(torch.abs(param.grad.data))
            count += 1
    if count > 0:
        mean = mean / count
    print(name)
    print(mean)

In [4]:
x = torch.rand(10, 10, dtype=torch.complex64)
fft2 = torch.fft.fft2(x)

In [6]:
fft2.shape

torch.Size([10, 10])

In [7]:
x

tensor([[0.4163+0.5822j, 0.6259+0.9470j, 0.4826+0.4858j, 0.0152+0.3946j,
         0.9043+0.2312j, 0.9522+0.2640j, 0.4039+0.4580j, 0.3752+0.3706j,
         0.4233+0.7891j, 0.4648+0.5390j],
        [0.8926+0.2537j, 0.9035+0.4851j, 0.4822+0.8309j, 0.9996+0.4227j,
         0.3947+0.5693j, 0.5234+0.4980j, 0.6035+0.1043j, 0.6450+0.1627j,
         0.0773+0.1069j, 0.3812+0.6207j],
        [0.6632+0.6174j, 0.3653+0.2913j, 0.4233+0.3574j, 0.7202+0.8177j,
         0.4895+0.1069j, 0.1272+0.0513j, 0.5123+0.2375j, 0.0678+0.2816j,
         0.1237+0.9714j, 0.7037+0.2694j],
        [0.3554+0.6205j, 0.9797+0.8925j, 0.2556+0.9361j, 0.9298+0.2993j,
         0.1719+0.4424j, 0.6964+0.5937j, 0.3072+0.8742j, 0.6552+0.0466j,
         0.1863+0.1210j, 0.9166+0.3598j],
        [0.1408+0.0401j, 0.7663+0.9466j, 0.8429+0.2753j, 0.5990+0.4808j,
         0.2602+0.2059j, 0.0450+0.2728j, 0.9249+0.3329j, 0.3924+0.3657j,
         0.4364+0.6914j, 0.4198+0.6828j],
        [0.9582+0.0750j, 0.0800+0.5483j, 0.6802+0.0511j, 0.7

In [ ]:
!pip install torch==1.8.0